In [280]:
#Loading required libraries
import pandas as pd
import sqlite3
from sqlite3 import Error
import os
from sqlalchemy import create_engine
import pandas as pd

In [281]:
conn_normalized = create_connection('normalized.db')

# Connect to the SQLite database
connection = sqlite3.connect('normalized.db')

# Create a cursor
cursor = connection.cursor()

# Query the sqlite_master table to get table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# Fetch and print the table names
tables = cursor.fetchall()
for table in tables:
    print(table[0])


product_40_code
product_30_code
product_20_code
product_10_code
Customer_details
basket_details
Store_details
transaction_details


In [282]:
def create_connection(db_file, delete_db=False):
    '''
        Purpose:
            To create database connection to retrieve tables
        Input: 
            db_file - name of the db file

        Output:
            conn - This establishes connection to the database

    '''    

    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn

        
def execute_sql_statement(sql_statement, conn):
    '''
        Input: 
            sql_statement - SQL statement to be executed
            conn - database on which the table will be created

        Output:
            rows - this returns the rows that's being created

    '''
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows


# Sample Data

### Transaction table

In [283]:
sql_statement= '''
SELECT
DISTINCT
*
FROM
transaction_details

'''


df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()

,RowNumber,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,CUST_CODE,BASKET_ID,STORE_CODE
0,1,200648,20070128,1,18,1,2.03,PRD0901825,CUST0000277824,994104200334249,STORE00703
1,2,200648,20070124,4,21,1,0.64,PRD0900801,CUST0000228256,994104200303399,STORE00996
2,3,200648,20070127,7,17,1,28.82,PRD0904300,CUST0000095739,994104200219916,STORE02450
3,4,200648,20070126,6,14,1,2.93,PRD0900123,CUST0000149372,994104200253687,STORE01113
4,5,200648,20070128,1,21,1,1.0,PRD0902029,CUST0000280589,994104200335936,STORE02653


### Store_details

In [284]:
sql_statement= '''
SELECT
DISTINCT
*
FROM
Store_details

'''


df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()

,STORE_CODE,STORE_FORMAT,STORE_REGION
0,STORE00703,LS,N03
1,STORE00996,LS,N02
2,STORE02450,LS,N02
3,STORE01113,LS,W02
4,STORE02653,LS,W02


### Customer_details

In [285]:
sql_statement= '''
SELECT
DISTINCT
*
FROM
Customer_details

'''


df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()

,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE
0,CUST0000277824,MM,OT
1,CUST0000228256,UM,OT
2,CUST0000095739,UM,OT
3,CUST0000149372,LA,YF
4,CUST0000280589,UM,YF


### product_10_code

In [286]:
sql_statement= '''
SELECT
DISTINCT
*
FROM
product_10_code

'''


df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()


,PROD_CODE,PROD_CODE_10
0,PRD0901825,CL00129
1,PRD0900801,CL00110
2,PRD0904300,CL00058
3,PRD0900123,CL00148
4,PRD0902029,CL00031


### product_20_code

In [287]:
sql_statement= '''
SELECT
DISTINCT
*
FROM
product_20_code

'''


df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()


,PROD_CODE_10,PROD_CODE_20
0,CL00129,DEP00046
1,CL00110,DEP00039
2,CL00058,DEP00016
3,CL00148,DEP00052
4,CL00031,DEP00008


### product_30_code

In [288]:
sql_statement= '''
SELECT
DISTINCT
*
FROM
product_30_code

'''


df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()


,PROD_CODE_20,PROD_CODE_30
0,DEP00046,G00013
1,DEP00039,G00010
2,DEP00016,G00006
3,DEP00052,G00015
4,DEP00008,G00004


### product_40_code

In [289]:
sql_statement= '''
SELECT
DISTINCT
*
FROM
product_40_code

'''


df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()


,PROD_CODE_30,PROD_CODE_40
0,G00013,D00003
1,G00010,D00003
2,G00006,D00002
3,G00015,D00003
4,G00004,D00002


### basket_details

In [290]:
sql_statement= '''
SELECT
DISTINCT
*
FROM
basket_details

'''


df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()


,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION
0,994104200334249,M,LA,Top Up,Mixed
1,994104200303399,L,UM,Top Up,Grocery
2,994104200219916,L,UM,Full Shop,Fresh
3,994104200253687,L,LA,Top Up,Mixed
4,994104200335936,L,MM,Top Up,Fresh


## 1. To check the number of customers in each customer bucket 

In [291]:
sql_statement= '''
SELECT
DISTINCT
CUST_PRICE_SENSITIVITY,
COUNT(DISTINCT CUST_CODE) AS COUNT
FROM
Customer_details
GROUP BY

CUST_PRICE_SENSITIVITY
'''


df = pd.read_sql_query(sql_statement, conn_normalized)
df


,CUST_PRICE_SENSITIVITY,COUNT
0,LA,986
1,MM,1562
2,UM,1099
3,XX,247


## 2. To check the number of customer in each basket domain

In [292]:
sql_statement= '''

SELECT 
DISTINCT
COUNT(DISTINCT CUST_CODE) AS COUNT,
BASKET_DOMINANT_MISSION

FROM transaction_details AS A
INNER JOIN basket_details AS B
ON A.BASKET_ID = B.BASKET_ID

GROUP BY 
B.BASKET_DOMINANT_MISSION

'''


df = pd.read_sql_query(sql_statement, conn_normalized)
df

,COUNT,BASKET_DOMINANT_MISSION
0,3524,Fresh
1,2806,Grocery
2,3247,Mixed
3,1640,Nonfood
4,319,XX


## 3. To check the number of transactions in each basket domain

In [293]:
sql_statement= '''

SELECT 
DISTINCT
COUNT(DISTINCT RowNumber) AS COUNT,
BASKET_DOMINANT_MISSION

FROM transaction_details AS A
LEFT JOIN basket_details AS B
ON A.BASKET_ID = B.BASKET_ID

GROUP BY 
B.BASKET_DOMINANT_MISSION

'''


df = pd.read_sql_query(sql_statement, conn_normalized)
df

,COUNT,BASKET_DOMINANT_MISSION
0,190756,Fresh
1,33590,Grocery
2,138971,Mixed
3,5790,Nonfood
4,503,XX


## 4. To check the no of transaction over months

In [294]:
sql_statement = '''
SELECT 
    COUNT(RowNumber) as count,
    Month
FROM (
    SELECT 
        RowNumber,
        CASE 
            WHEN CAST(SUBSTRING(SHOP_DATE, 5, 2) AS INT) = 1 THEN 'January'
            WHEN CAST(SUBSTRING(SHOP_DATE, 5, 2) AS INT) = 2 THEN 'February'
            WHEN CAST(SUBSTRING(SHOP_DATE, 5, 2) AS INT) = 3 THEN 'March'
            WHEN CAST(SUBSTRING(SHOP_DATE, 5, 2) AS INT) = 4 THEN 'April'
            WHEN CAST(SUBSTRING(SHOP_DATE, 5, 2) AS INT) = 5 THEN 'May'
            WHEN CAST(SUBSTRING(SHOP_DATE, 5, 2) AS INT) = 6 THEN 'June'
            WHEN CAST(SUBSTRING(SHOP_DATE, 5, 2) AS INT) = 7 THEN 'July'
            WHEN CAST(SUBSTRING(SHOP_DATE, 5, 2) AS INT) = 8 THEN 'August'
            WHEN CAST(SUBSTRING(SHOP_DATE, 5, 2) AS INT) = 9 THEN 'September'
            WHEN CAST(SUBSTRING(SHOP_DATE, 5, 2) AS INT) = 10 THEN 'October'
            WHEN CAST(SUBSTRING(SHOP_DATE, 5, 2) AS INT) = 11 THEN 'November'
            WHEN CAST(SUBSTRING(SHOP_DATE, 5, 2) AS INT) = 12 THEN 'December'
        END AS Month
    FROM transaction_details
) AS MonthlyData
GROUP BY 
    Month
ORDER BY Month;
'''

df = pd.read_sql_query(sql_statement, conn_normalized)
df


,count,Month
0,24197,April
1,35036,August
2,35545,December
3,29080,February
4,35846,January
5,35036,July
6,34168,June
7,35634,May
8,34745,November
9,35896,October


## 5. To check no of revenue under each category of product 

In [295]:
sql_statement= '''

SELECT 
DISTINCT
SUM(AMOUNT) AS TOTAL_REVENUE,
PROD_CODE_40
FROM
(
    SELECT 
    DISTINCT 
    A.*,
    B.*,
    C.*,
    D.*,
    E.*,
    QUANTITY * SPEND AS AMOUNT

    FROM transaction_details AS A
    LEFT JOIN product_10_code AS B
    ON A.PROD_CODE = B.PROD_CODE
    LEFT JOIN product_20_code AS C
    ON B.PROD_CODE_10 = C.PROD_CODE_10
    LEFT JOIN product_30_code AS D
    ON C.PROD_CODE_20 = D.PROD_CODE_20
    LEFT JOIN product_40_code AS E
    ON D.PROD_CODE_30 = E.PROD_CODE_30

)
GROUP BY 
PROD_CODE_40
ORDER BY TOTAL_REVENUE DESC
'''


df = pd.read_sql_query(sql_statement, conn_normalized)
df

,TOTAL_REVENUE,PROD_CODE_40
0,668810.97,D00002
1,367741.34,D00003
2,183605.37,D00008
3,103431.74,D00005
4,91808.62,D00001
5,15071.72,D00004
6,6150.53,D00009
7,5818.89,D00006
8,290.16,D00007


## 6. To check the total revenue distribution underthe top most PROD_CODE_40 category

In [296]:
sql_statement= '''

WITH top_table as 
(
    SELECT 
    DISTINCT
    SUM(AMOUNT) AS TOTAL_REVENUE,
    PROD_CODE_40
    FROM
    (
        SELECT 
        DISTINCT 
        A.PROD_CODE,
        B.PROD_CODE_10,
        C.PROD_CODE_20,
        D.PROD_CODE_30,
        E.PROD_CODE_40,
        QUANTITY * SPEND AS AMOUNT

        FROM transaction_details AS A
        LEFT JOIN product_10_code AS B
        ON A.PROD_CODE = B.PROD_CODE
        LEFT JOIN product_20_code AS C
        ON B.PROD_CODE_10 = C.PROD_CODE_10
        LEFT JOIN product_30_code AS D
        ON C.PROD_CODE_20 = D.PROD_CODE_20
        LEFT JOIN product_40_code AS E
        ON D.PROD_CODE_30 = E.PROD_CODE_30

    )
    GROUP BY 
    PROD_CODE_40
    ORDER BY TOTAL_REVENUE DESC
)

SELECT
DISTINCT 
SUM(AMOUNT) AS TOTAL_REVENUE,
PROD_CODE_30,
PROD_CODE_40
FROM
(
SELECT 
DISTINCT 
QUANTITY * SPEND AS AMOUNT,
E.PROD_CODE_30,
E.PROD_CODE_40
FROM 
transaction_details AS A
LEFT JOIN product_10_code AS B
ON A.PROD_CODE = B.PROD_CODE
LEFT JOIN product_20_code AS C
ON B.PROD_CODE_10 = C.PROD_CODE_10
LEFT JOIN product_30_code AS D
ON C.PROD_CODE_20 = D.PROD_CODE_20
LEFT JOIN product_40_code AS E
ON D.PROD_CODE_30 = E.PROD_CODE_30
WHERE E.PROD_CODE_40 = (SELECT PROD_CODE_40 FROM top_table WHERE TOTAL_REVENUE = (SELECT MAX(TOTAL_REVENUE) FROM top_table))
)
GROUP BY PROD_CODE_30
ORDER BY TOTAL_REVENUE DESC


'''


df = pd.read_sql_query(sql_statement, conn_normalized)
df

,TOTAL_REVENUE,PROD_CODE_30,PROD_CODE_40
0,50197.40,G00007,D00002
1,13652.09,G00008,D00002
2,13291.74,G00004,D00002
3,9875.75,G00005,D00002
4,9115.32,G00006,D00002
5,4776.05,G00009,D00002


## 7. Time wise total revenue 

In [297]:
sql_statement = '''
SELECT 
DISTINCT 
SUM(AMOUNT) AS TOTAL_AMOUNT,
TimePeriod

FROM 
(
    SELECT 
        DISTINCT
        AMOUNT,
        CASE 
            WHEN SHOP_HOUR BETWEEN 1 AND 6 THEN 'Night'
            WHEN SHOP_HOUR BETWEEN 7 AND 12 THEN 'Morning'
            WHEN SHOP_HOUR BETWEEN 13 AND 18 THEN 'Afternoon'
            WHEN SHOP_HOUR BETWEEN 19 AND 24 THEN 'Evening'
        END AS TimePeriod
    FROM
    (
        SELECT
            DISTINCT
            QUANTITY * SPEND AS AMOUNT,
            CAST(SHOP_HOUR AS INT) AS SHOP_HOUR
        FROM
            transaction_details
    )
)
GROUP BY
TimePeriod;
'''

df = pd.read_sql_query(sql_statement, conn_normalized)
df


,TOTAL_AMOUNT,TimePeriod
0,114595.41,Afternoon
1,59112.51,Evening
2,117319.72,Morning


## 8. To check which region sold the most revenue

In [298]:
sql_statement = '''
SELECT 
DISTINCT 
SUM(QUANTITY * SPEND) AS TOTAL_REVENUE,
STORE_REGION

FROM transaction_details AS A 
LEFT JOIN Store_details AS B 
ON A.STORE_CODE = B.STORE_CODE

GROUP BY STORE_REGION
ORDER BY TOTAL_REVENUE DESC
;
'''

df = pd.read_sql_query(sql_statement, conn_normalized)
df

,TOTAL_REVENUE,STORE_REGION
0,158657.45,W02
1,141788.01,S02
2,133542.37,N01
3,129390.82,N03
4,128137.09,N02
5,120449.62,S03
6,119927.03,E02
7,108440.21,E03
8,105830.82,E01
9,102774.07,S01


## 9. To check the total spend of aggregated on customer lifestage, store region, and basket dominant mission

In [299]:
sql_statement ='''

SELECT
    C.CUST_LIFESTAGE,
    D.STORE_REGION,
    E.BASKET_DOMINANT_MISSION,
    SUM(A.SPEND) AS TOTAL_SPEND
FROM
    transaction_details AS A
LEFT JOIN Customer_details AS C ON A.CUST_CODE = C.CUST_CODE
LEFT JOIN Store_details AS D ON A.STORE_CODE = D.STORE_CODE
LEFT JOIN basket_details AS E ON A.BASKET_ID = E.BASKET_ID
GROUP BY
    C.CUST_LIFESTAGE,
    D.STORE_REGION,
    E.BASKET_DOMINANT_MISSION
ORDER BY
TOTAL_SPEND
    ;
'''

df = pd.read_sql_query(f" {sql_statement}", conn_normalized)
df.head()

,CUST_LIFESTAGE,STORE_REGION,BASKET_DOMINANT_MISSION,TOTAL_SPEND
0,OF,W01,XX,0.01
1,YA,E02,XX,0.01
2,OF,N01,XX,0.02
3,PE,W03,XX,0.02
4,PE,E02,XX,0.03


## 10. To understand the customer transactions, their life stages, store regions, basket missions, and specific product codes during a specific date range. The SUM_QUANTITY column reflects the aggregated quantity of products purchased in each basket.

In [300]:
sql_statement ='''

SELECT 
*
FROM 
(
SELECT
    A.CUST_CODE,
    B.CUST_LIFESTAGE,
    C.STORE_REGION,
    D.BASKET_DOMINANT_MISSION,
    E.PROD_CODE_40,
    F.BASKET_SIZE,
    G.SUM_QUANTITY,
    SUBSTR(SHOP_DATE, 1, 4) || '-' || SUBSTR(SHOP_DATE, 5, 2) || '-' || SUBSTR(SHOP_DATE, 7, 2) AS SHOP_DATE
FROM
    transaction_details AS A
LEFT JOIN Customer_details AS B ON A.CUST_CODE = B.CUST_CODE
LEFT JOIN Store_details AS C ON A.STORE_CODE = C.STORE_CODE
LEFT JOIN basket_details AS D ON A.BASKET_ID = D.BASKET_ID
LEFT JOIN product_40_code AS E ON A.PROD_CODE = E.PROD_CODE_40
LEFT JOIN basket_details AS F ON A.BASKET_ID = F.BASKET_ID
LEFT JOIN (
    SELECT BASKET_ID, SUM(QUANTITY) AS SUM_QUANTITY
    FROM transaction_details
    GROUP BY BASKET_ID
) AS G ON A.BASKET_ID = G.BASKET_ID
)
WHERE 
DATE(SHOP_DATE) >= '2007-02-01'
AND 
DATE(SHOP_DATE) <= '2007-12-31'

ORDER BY
    SUM_QUANTITY DESC;
'''

df = pd.read_sql_query(f" {sql_statement}", conn_normalized)
df.head()





,CUST_CODE,CUST_LIFESTAGE,STORE_REGION,BASKET_DOMINANT_MISSION,PROD_CODE_40,BASKET_SIZE,SUM_QUANTITY,SHOP_DATE
0,CUST0000864337,OT,S02,Mixed,None,L,299,2007-02-13
1,CUST0000864337,OT,S02,Mixed,None,L,299,2007-02-13
2,CUST0000864337,OT,S02,Mixed,None,L,299,2007-02-13
3,CUST0000864337,OT,S02,Mixed,None,L,299,2007-02-13
4,CUST0000864337,OT,S02,Mixed,None,L,299,2007-02-13


## 11. To update the Customer's (CUST0000277824) customer lifestage 

In [301]:
import pandas as pd

print("\033[1mBefore Updation\033[0m")
sql_statement_before = '''
SELECT 
    DISTINCT 
    CUST_CODE,
    CUST_LIFESTAGE
FROM 
    Customer_details
WHERE 
    CUST_CODE = 'CUST0000149372';
'''

df_before = pd.read_sql_query(sql_statement_before, conn_normalized)
print(df_before)

# Execute the update query
sql_statement_update = '''
UPDATE Customer_details
SET CUST_LIFESTAGE = 'OT'
WHERE CUST_CODE = 'CUST0000149372';
'''

conn_normalized.execute(sql_statement_update)

print("\033[1mAfter Updation\033[0m")
sql_statement_after = '''
SELECT 
    DISTINCT 
    CUST_CODE,
    CUST_LIFESTAGE
FROM 
    Customer_details
WHERE 
    CUST_CODE = 'CUST0000149372';
'''

df_after = pd.read_sql_query(sql_statement_after, conn_normalized)
print(df_after)


Before Updation
        CUST_CODE CUST_LIFESTAGE
0  CUST0000149372             YF
After Updation
        CUST_CODE CUST_LIFESTAGE
0  CUST0000149372             OT


## 12. Deleting all data from a particular store (STORE00703)

In [302]:
## Deleting the records in order from the tables as per foreign key relationships

## 1. Delete from transaction_details tables 
## 2. Then delete from the store_details table

print("\033[1mBefore Deletion\033[0m")
sql_statement_before = '''
SELECT 
   *
FROM 
    Store_details
WHERE 
    STORE_CODE = 'STORE00703';
'''

df_before = pd.read_sql_query(sql_statement_before, conn_normalized)
print(df_before)

# Execute the delete query
sql_statement_update = '''
DELETE FROM transaction_details WHERE STORE_CODE = 'STORE00703';
'''

conn_normalized.execute(sql_statement_update)


sql_statement_update = '''
DELETE FROM Store_details WHERE STORE_CODE = 'STORE00703';
'''

conn_normalized.execute(sql_statement_update)


print("\033[1mAfter  Deletion\033[0m")
sql_statement_before = '''
SELECT 
   *
FROM Store_details AS A
LEFT JOIN transaction_details AS B 
ON A.STORE_CODE = B.STORE_CODE
WHERE 
    A.STORE_CODE = 'STORE00703';
'''

df_before = pd.read_sql_query(sql_statement_before, conn_normalized)
print(df_before)


Before Deletion
   STORE_CODE STORE_FORMAT STORE_REGION
0  STORE00703           LS          N03
After  Deletion
Empty DataFrame
Columns: [STORE_CODE, STORE_FORMAT, STORE_REGION, RowNumber, SHOP_WEEK, SHOP_DATE, SHOP_WEEKDAY, SHOP_HOUR, QUANTITY, SPEND, PROD_CODE, CUST_CODE, BASKET_ID, STORE_CODE]
Index: []


## 13. To check revenue for each store type

In [303]:
sql_statement = '''
SELECT 
DISTINCT 
SUM(QUANTITY * SPEND) AS TOTAL_REVENUE,
STORE_FORMAT

FROM transaction_details AS A 
LEFT JOIN Store_details AS B 
ON A.STORE_CODE = B.STORE_CODE

GROUP BY STORE_FORMAT
ORDER BY TOTAL_REVENUE DESC
;
'''

df = pd.read_sql_query(sql_statement, conn_normalized)
df

,TOTAL_REVENUE,STORE_FORMAT
0,958756.39,LS
1,293986.01,MS
2,116533.15,XLS
3,70785.08,SS


## 14. To insert a new transaction record of our newest product into the respective tables

In [304]:
print("\033[1mBefore Insertion\033[0m")
sql_statement_before='''
    SELECT 
    DISTINCT
    SUM(AMOUNT) AS TOTAL_REVENUE,
    PROD_CODE_10,
    PROD_CODE_20,
    PROD_CODE_30,
    PROD_CODE_40
    FROM
    (
        SELECT 
        DISTINCT 
        A.PROD_CODE,
        B.PROD_CODE_10,
        C.PROD_CODE_20,
        D.PROD_CODE_30,
        E.PROD_CODE_40,
        QUANTITY * SPEND AS AMOUNT

        FROM transaction_details AS A
        LEFT JOIN product_10_code AS B
        ON A.PROD_CODE = B.PROD_CODE
        LEFT JOIN product_20_code AS C
        ON B.PROD_CODE_10 = C.PROD_CODE_10
        LEFT JOIN product_30_code AS D
        ON C.PROD_CODE_20 = D.PROD_CODE_20
        LEFT JOIN product_40_code AS E
        ON D.PROD_CODE_30 = E.PROD_CODE_30

    )
    GROUP BY 
    PROD_CODE_10,
    PROD_CODE_20,
    PROD_CODE_30,
    PROD_CODE_40
    ORDER BY PROD_CODE_40 DESC

'''

df_after = pd.read_sql_query(sql_statement_before, conn_normalized)
df_after

Before Insertion


,TOTAL_REVENUE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40
0,154.94,CL00248,DEP00089,G00030,D00009
1,2249.21,CL00249,DEP00089,G00030,D00009
2,4.77,CL00250,DEP00090,G00031,D00009
3,8467.28,CL00229,DEP00081,G00027,D00008
4,1575.92,CL00230,DEP00081,G00027,D00008
...,...,...,...,...,...
243,200.67,CL00018,DEP00005,G00003,D00001
244,42.03,CL00019,DEP00005,G00003,D00001
245,1791.59,CL00020,DEP00005,G00003,D00001
246,85.96,CL00021,DEP00006,G00003,D00001


In [305]:
from sqlite3 import IntegrityError  # Assuming you're using SQLite

def insert_degree(conn, values):
    try:
        sql = '''INSERT INTO product_40_code (PROD_CODE_30, PROD_CODE_40)
                 VALUES(?, ?)'''
        cur = conn.cursor()
        cur.execute(sql, values)
        return cur.lastrowid
    except IntegrityError as e:
        pass

prod_40 = [('G09000', 'D90000')]

# Assuming conn_normalized is properly defined somewhere in your code
with conn_normalized:
    for record in prod_40:
        insert_degree(conn_normalized, record)



## Update the product 30 table
def insert_degree(conn, values):
    try:
        sql = ''' INSERT INTO product_30_code (PROD_CODE_20, PROD_CODE_30)
                  VALUES(?,?) '''
        cur = conn.cursor()
        cur.execute(sql, values)
        return cur.lastrowid
    except IntegrityError as e:
        pass

prod_30 = [('DEP00900','G09000')]

with conn_normalized:
    for record in prod_30:
        insert_degree(conn_normalized, record)


## Update the product 20 table
def insert_degree(conn, values):
    try:
        sql = ''' INSERT INTO product_20_code (PROD_CODE_10, PROD_CODE_20)
                  VALUES(?,?) '''
        cur = conn.cursor()
        cur.execute(sql, values)
        return cur.lastrowid
    except IntegrityError as e:
        pass
    
    
prod_20 = [('CL00999','DEP00900')]

with conn_normalized:
    for record in prod_20:
        insert_degree(conn_normalized, record)


## Update the product 10 table
def insert_degree(conn, values):
    try:
        sql = ''' INSERT INTO product_10_code (PROD_CODE, PROD_CODE_10)
                  VALUES(?,?) '''
        cur = conn.cursor()
        cur.execute(sql, values)
        return cur.lastrowid
    except IntegrityError as e:
        pass

prod_10 = [('PRD0905555','CL00999')]

with conn_normalized:
    for record in prod_10:
        insert_degree(conn_normalized, record)
        
        
## Update the transaction table       
def insert_td(conn, values):
    try:
        sql = ''' INSERT INTO transaction_details (SHOP_WEEK, SHOP_DATE, SHOP_WEEKDAY, SHOP_HOUR, QUANTITY, SPEND, PROD_CODE, CUST_CODE, BASKET_ID, STORE_CODE)
                  VALUES(?,?,?,?,?,?,?,?,?,?) '''
        cur = conn.cursor()

        cur.execute(sql, values)
        return cur.lastrowid
    except IntegrityError as e:
        pass

records_to_be_inserted = [(200648,20070129,1,18,1,2.03,'PRD0905555','CUST0000277824',994104200334249,'STORE00703')]

with conn_normalized:
    for record in records_to_be_inserted:
        insert_td(conn_normalized, record)

In [306]:
print("\033[1mAfter Insertion\033[0m")
sql_statement_before='''
    SELECT 
    DISTINCT
    SUM(AMOUNT) AS TOTAL_REVENUE,
    PROD_CODE_10,
    PROD_CODE_20,
    PROD_CODE_30,
    PROD_CODE_40
    FROM
    (
        SELECT 
        DISTINCT 
        A.PROD_CODE,
        B.PROD_CODE_10,
        C.PROD_CODE_20,
        D.PROD_CODE_30,
        E.PROD_CODE_40,
        QUANTITY * SPEND AS AMOUNT

        FROM transaction_details AS A
        LEFT JOIN product_10_code AS B
        ON A.PROD_CODE = B.PROD_CODE
        LEFT JOIN product_20_code AS C
        ON B.PROD_CODE_10 = C.PROD_CODE_10
        LEFT JOIN product_30_code AS D
        ON C.PROD_CODE_20 = D.PROD_CODE_20
        LEFT JOIN product_40_code AS E
        ON D.PROD_CODE_30 = E.PROD_CODE_30

    )
    GROUP BY 
    PROD_CODE_10,
    PROD_CODE_20,
    PROD_CODE_30,
    PROD_CODE_40
    ORDER BY PROD_CODE_40 DESC

'''

df_after = pd.read_sql_query(sql_statement_before, conn_normalized)
df_after

After Insertion


,TOTAL_REVENUE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40
0,154.94,CL00248,DEP00089,G00030,D00009
1,2249.21,CL00249,DEP00089,G00030,D00009
2,4.77,CL00250,DEP00090,G00031,D00009
3,8467.28,CL00229,DEP00081,G00027,D00008
4,1575.92,CL00230,DEP00081,G00027,D00008
...,...,...,...,...,...
243,200.67,CL00018,DEP00005,G00003,D00001
244,42.03,CL00019,DEP00005,G00003,D00001
245,1791.59,CL00020,DEP00005,G00003,D00001
246,85.96,CL00021,DEP00006,G00003,D00001


# Identifying the problematic queries

## Query 1

## Unoptimised Query

Existing Problem:

- Redundant use of DISTINCT in the subquery, possibly impacting performance.
- Unnecessary selection of columns in the subquery, introducing complexity without adding value.

In [307]:
sql_statement = '''

SELECT DISTINCT
    SUM(AMOUNT) AS TOTAL_REVENUE,
    PROD_CODE_40
FROM (
    SELECT DISTINCT 
        A.*,
        B.*,
        C.*,
        D.*,
        E.*,
        QUANTITY * SPEND AS AMOUNT
    FROM transaction_details AS A
    LEFT JOIN product_10_code AS B ON A.PROD_CODE = B.PROD_CODE
    LEFT JOIN product_20_code AS C ON B.PROD_CODE_10 = C.PROD_CODE_10
    LEFT JOIN product_30_code AS D ON C.PROD_CODE_20 = D.PROD_CODE_20
    LEFT JOIN product_40_code AS E ON D.PROD_CODE_30 = E.PROD_CODE_30
)
GROUP BY PROD_CODE_40
ORDER BY TOTAL_REVENUE DESC;


'''



df = pd.read_sql_query(f"EXPLAIN{sql_statement}", conn_normalized)
# Define cost estimates for each opcode
cost_estimates = {
    "Init": 1,
    "InitCoroutine": 1,
    "Null": 0,
    "OpenRead": 10,
    "OpenScan": 10,
    "ResultRow": 1,
    "Sort": 100,
    "SortKey": 1,
    "Aggregate": 5,
    "Filter": 5,
    "Project": 1,
    "HashJoin": 100,
    "MergeJoin": 100,
    "NestedLoopJoin": 100,
}

# Calculate the cost for each row
df["cost"] = df["opcode"].apply(lambda opcode: cost_estimates.get(opcode, 0))

# Calculate the overall cost
total_cost = df["cost"].sum()

# Print the overall cost
print(f"\033[1mTotal cost:\033[0m {total_cost}")


print("\033[1mExecution Plan\033[0m")

# Connect to the SQLite database
conn = sqlite3.connect('normalized.db')
cursor = conn.cursor()

# Execute EXPLAIN QUERY PLAN
explain_query = f"EXPLAIN QUERY PLAN {sql_statement}"
cursor.execute(explain_query)

# Fetch and print the execution plan
execution_plan = cursor.fetchall()
for plan in execution_plan:
    print(plan)

# Close the cursor and connection
cursor.close()
conn.close()

import time
# Connect to the SQLite database
conn = sqlite3.connect('normalized.db')
cursor = conn.cursor()

# Start timing
start_time = time.time()

cursor.execute(sql_statement)

# Get all results
results = cursor.fetchall()

# Stop timing
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time

# Close the cursor and connection
cursor.close()
conn.close()

# Print the execution time
print("\033[1mExecution time:\033[0m", execution_time, "seconds")



Total cost: 94
Execution Plan
(2, 0, 0, 'CO-ROUTINE (subquery-1)')
(13, 2, 0, 'SCAN A')
(15, 2, 0, 'SEARCH B USING INDEX sqlite_autoindex_product_10_code_1 (PROD_CODE=?) LEFT-JOIN')
(22, 2, 0, 'SEARCH C USING INDEX sqlite_autoindex_product_20_code_1 (PROD_CODE_10=?) LEFT-JOIN')
(30, 2, 0, 'SEARCH D USING INDEX sqlite_autoindex_product_30_code_1 (PROD_CODE_20=?) LEFT-JOIN')
(38, 2, 0, 'SEARCH E USING INDEX sqlite_autoindex_product_40_code_1 (PROD_CODE_30=?) LEFT-JOIN')
(114, 0, 0, 'SCAN (subquery-1)')
(117, 0, 0, 'USE TEMP B-TREE FOR GROUP BY')
(157, 0, 0, 'USE TEMP B-TREE FOR DISTINCT')
(158, 0, 0, 'USE TEMP B-TREE FOR ORDER BY')
Execution time: 1.1076397895812988 seconds


1. `(2, 0, 0, 'CO-ROUTINE (subquery-1)')`: This step suggests the initiation of a co-routine, possibly associated with a subquery (subquery-1).

2. `(13, 2, 0, 'SCAN A')`: This indicates a table scan on a table referred to as A. It's fetching rows from the specified table.

3. `(15, 2, 0, 'SEARCH B USING INDEX sqlite_autoindex_product_10_code_1 (PROD_CODE=?) LEFT-JOIN')`: This step involves searching in table B using an index (sqlite_autoindex_product_10_code_1) on the column `PROD_CODE`. It appears to be part of a LEFT JOIN operation.

4. `(22, 2, 0, 'SEARCH C USING INDEX sqlite_autoindex_product_20_code_1 (PROD_CODE_10=?) LEFT-JOIN')`: Similar to the previous step, this involves searching in table C using an index (sqlite_autoindex_product_20_code_1) on the column `PROD_CODE_10`.

5. `(30, 2, 0, 'SEARCH D USING INDEX sqlite_autoindex_product_30_code_1 (PROD_CODE_20=?) LEFT-JOIN')`: Similar to the previous steps, this involves searching in table D using an index (sqlite_autoindex_product_30_code_1) on the column `PROD_CODE_20`.

6. `(38, 2, 0, 'SEARCH E USING INDEX sqlite_autoindex_product_40_code_1 (PROD_CODE_30=?) LEFT-JOIN')`: This step involves searching in table E using an index (sqlite_autoindex_product_40_code_1) on the column `PROD_CODE_30`.

7. `(114, 0, 0, 'SCAN (subquery-1)')`: Another table scan, possibly part of a subquery (subquery-1).

8. `(117, 0, 0, 'USE TEMP B-TREE FOR GROUP BY')`: Temporary B-tree usage for the GROUP BY operation.

9. `(157, 0, 0, 'USE TEMP B-TREE FOR DISTINCT')`: Temporary B-tree usage for DISTINCT operation.

10. `(158, 0, 0, 'USE TEMP B-TREE FOR ORDER BY')`: Temporary B-tree usage for ORDER BY operation.



## Optimized Query

Solution:

- Removed the redundant use of DISTINCT in the subquery for clarity and potential performance improvement.
- Simplified the column selection in the outer query for better readability and reduced complexity.

In [308]:
sql_statement = '''
 
SELECT 
    PROD_CODE_40,
    SUM(QUANTITY * SPEND) AS TOTAL_REVENUE
FROM
    transaction_details AS A
LEFT JOIN product_10_code AS B ON A.PROD_CODE = B.PROD_CODE
LEFT JOIN product_20_code AS C ON B.PROD_CODE_10 = C.PROD_CODE_10
LEFT JOIN product_30_code AS D ON C.PROD_CODE_20 = D.PROD_CODE_20
LEFT JOIN product_40_code AS E ON D.PROD_CODE_30 = E.PROD_CODE_30
GROUP BY 
    PROD_CODE_40
ORDER BY 
    TOTAL_REVENUE DESC;


'''

df = pd.read_sql_query(f"EXPLAIN{sql_statement}", conn_normalized)

# Calculate the cost for each row
df["cost"] = df["opcode"].apply(lambda opcode: cost_estimates.get(opcode, 0))

# Calculate the overall cost
total_cost = df["cost"].sum()

# Print the overall cost
print(f"\033[1mTotal cost:\033[0m {total_cost}")



print(f"\033[1mExecution Plan\033[0m")


sql_statement = '''
SELECT 
    PROD_CODE_40,
    SUM(QUANTITY * SPEND) AS TOTAL_REVENUE
FROM
    transaction_details AS A
LEFT JOIN product_10_code AS B ON A.PROD_CODE = B.PROD_CODE
LEFT JOIN product_20_code AS C ON B.PROD_CODE_10 = C.PROD_CODE_10
LEFT JOIN product_30_code AS D ON C.PROD_CODE_20 = D.PROD_CODE_20
LEFT JOIN product_40_code AS E ON D.PROD_CODE_30 = E.PROD_CODE_30
GROUP BY 
    PROD_CODE_40
ORDER BY 
    TOTAL_REVENUE DESC;
'''

# Connect to the SQLite database
conn = sqlite3.connect('normalized.db')
cursor = conn.cursor()

# Execute EXPLAIN QUERY PLAN
explain_query = f"EXPLAIN QUERY PLAN {sql_statement}"
cursor.execute(explain_query)

# Fetch and print the execution plan
execution_plan = cursor.fetchall()
for plan in execution_plan:
    print(plan)

# Close the cursor and connection
cursor.close()
conn.close()

print(f"\033[1mExecution Time\033[0m")

import time
# Connect to the SQLite database
conn = sqlite3.connect('normalized.db')
cursor = conn.cursor()

# Start timing
start_time = time.time()

cursor.execute(sql_statement)

# Get all results
results = cursor.fetchall()

# Stop timing
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time

# Close the cursor and connection
cursor.close()
conn.close()

# Print the execution time
print("Execution time:", execution_time, "seconds")


Total cost: 92
Execution Plan
(15, 0, 0, 'SCAN A')
(17, 0, 0, 'SEARCH B USING INDEX sqlite_autoindex_product_10_code_1 (PROD_CODE=?) LEFT-JOIN')
(24, 0, 0, 'SEARCH C USING INDEX sqlite_autoindex_product_20_code_1 (PROD_CODE_10=?) LEFT-JOIN')
(32, 0, 0, 'SEARCH D USING INDEX sqlite_autoindex_product_30_code_1 (PROD_CODE_20=?) LEFT-JOIN')
(40, 0, 0, 'SEARCH E USING INDEX sqlite_autoindex_product_40_code_1 (PROD_CODE_30=?) LEFT-JOIN')
(48, 0, 0, 'USE TEMP B-TREE FOR GROUP BY')
(104, 0, 0, 'USE TEMP B-TREE FOR ORDER BY')
Execution Time
Execution time: 0.8128397464752197 seconds


1. `(15, 0, 0, 'SCAN A')`: This step indicates a table scan on a table referred to as A. It involves fetching rows directly from the table.

2. `(17, 0, 0, 'SEARCH B USING INDEX sqlite_autoindex_product_10_code_1 (PROD_CODE=?) LEFT-JOIN')`: This involves searching in table B using an index (sqlite_autoindex_product_10_code_1) on the column `PROD_CODE`. It appears to be part of a LEFT JOIN operation.

3. `(24, 0, 0, 'SEARCH C USING INDEX sqlite_autoindex_product_20_code_1 (PROD_CODE_10=?) LEFT-JOIN')`: Similar to the previous step, this involves searching in table C using an index (sqlite_autoindex_product_20_code_1) on the column `PROD_CODE_10`.

4. `(32, 0, 0, 'SEARCH D USING INDEX sqlite_autoindex_product_30_code_1 (PROD_CODE_20=?) LEFT-JOIN')`: Similar to the previous steps, this involves searching in table D using an index (sqlite_autoindex_product_30_code_1) on the column `PROD_CODE_20`.

5. `(40, 0, 0, 'SEARCH E USING INDEX sqlite_autoindex_product_40_code_1 (PROD_CODE_30=?) LEFT-JOIN')`: This step involves searching in table E using an index (sqlite_autoindex_product_40_code_1) on the column `PROD_CODE_30`.

6. `(48, 0, 0, 'USE TEMP B-TREE FOR GROUP BY')`: Temporary B-tree usage for the GROUP BY operation. This indicates that a temporary data structure is used to facilitate the grouping operation.

7. `(104, 0, 0, 'USE TEMP B-TREE FOR ORDER BY')`: Temporary B-tree usage for the ORDER BY operation. This indicates that a temporary data structure is used to facilitate the sorting operation.




### - This optimzation has both reduced time and cost
### - This has a reduced execution time to 26.51%
### - It has reduced cost from 94 to 92

## Query 2

### Unoptimized query

Existing problem:
   - The unoptimized query suffers from performance issues due to excessive joins involving multiple tables (transaction_details, Customer_details, Store_details, and basket_details) without proper indexing
   - This can result in high computational cost and inefficient data retrieval, especially with large datasets
   - The lack of aggregation at an early stage further contributes to the query's suboptimal performance.
    
    

In [309]:
sql_statement ='''

SELECT
    C.CUST_LIFESTAGE,
    D.STORE_REGION,
    E.BASKET_DOMINANT_MISSION,
    SUM(A.SPEND) AS TOTAL_SPEND
FROM
    transaction_details AS A
LEFT JOIN Customer_details AS C ON A.CUST_CODE = C.CUST_CODE
LEFT JOIN Store_details AS D ON A.STORE_CODE = D.STORE_CODE
LEFT JOIN basket_details AS E ON A.BASKET_ID = E.BASKET_ID
GROUP BY
    C.CUST_LIFESTAGE,
    D.STORE_REGION,
    E.BASKET_DOMINANT_MISSION;
'''

df = pd.read_sql_query(f"EXPLAIN {sql_statement}", conn_normalized)

# Calculate the cost for each row
df["cost"] = df["opcode"].apply(lambda opcode: cost_estimates.get(opcode, 0))

# Calculate the overall cost
total_cost = df["cost"].sum()

# Print the overall cost
print(f"\033[1mTotal cost:\033[0m {total_cost}")

print("\033[1mExecution Plan\033[0m")

# Connect to the SQLite database
conn = sqlite3.connect('normalized.db')
cursor = conn.cursor()

# Execute EXPLAIN QUERY PLAN
explain_query = f"EXPLAIN QUERY PLAN {sql_statement}"
cursor.execute(explain_query)

# Fetch and print the execution plan
execution_plan = cursor.fetchall()
for plan in execution_plan:
    print(plan)

# Close the cursor and connection
cursor.close()
conn.close()


print("\033[1mExecution Time\033[0m")

# Connect to the SQLite database
conn = sqlite3.connect('normalized.db')
cursor = conn.cursor()

# Start timing
start_time = time.time()

cursor.execute(sql_statement)

# Get all results
results = cursor.fetchall()

# Stop timing
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time

# Close the cursor and connection
cursor.close()
conn.close()

# Print the execution time
print("Execution time:", execution_time, "seconds")


Total cost: 72
Execution Plan
(12, 0, 0, 'SCAN A')
(14, 0, 0, 'SEARCH C USING INDEX sqlite_autoindex_Customer_details_1 (CUST_CODE=?) LEFT-JOIN')
(21, 0, 0, 'SEARCH D USING INDEX sqlite_autoindex_Store_details_1 (STORE_CODE=?) LEFT-JOIN')
(28, 0, 0, 'SEARCH E USING INDEX sqlite_autoindex_basket_details_1 (BASKET_ID=?) LEFT-JOIN')
(35, 0, 0, 'USE TEMP B-TREE FOR GROUP BY')
Execution Time
Execution time: 1.4913511276245117 seconds


1. `(12, 0, 0, 'SCAN A')`: This step indicates a table scan on a table referred to as A. It involves fetching rows directly from the table.

2. `(14, 0, 0, 'SEARCH C USING INDEX sqlite_autoindex_Customer_details_1 (CUST_CODE=?) LEFT-JOIN')`: This step involves searching in table C (Customer_details) using an index (sqlite_autoindex_Customer_details_1) on the column `CUST_CODE`. It appears to be part of a LEFT JOIN operation.

3. `(21, 0, 0, 'SEARCH D USING INDEX sqlite_autoindex_Store_details_1 (STORE_CODE=?) LEFT-JOIN')`: Similar to the previous step, this involves searching in table D (Store_details) using an index (sqlite_autoindex_Store_details_1) on the column `STORE_CODE`.

4. `(28, 0, 0, 'SEARCH E USING INDEX sqlite_autoindex_basket_details_1 (BASKET_ID=?) LEFT-JOIN')`: Similar to the previous steps, this involves searching in table E (basket_details) using an index (sqlite_autoindex_basket_details_1) on the column `BASKET_ID`.

5. `(35, 0, 0, 'USE TEMP B-TREE FOR GROUP BY')`: Temporary B-tree usage for the GROUP BY operation. This indicates that a temporary data structure is used to facilitate the grouping operation.


### Optimized Query

Solution:
   - In this optimized query, the subquery aggregates the spend at the transaction level first, reducing the data to be joined
   - This can often improve performance, especially when dealing with large datasets
   - Also, ensure that appropriate indexes are in place on the columns involved in JOIN and WHERE clauses.
    
    
    

In [310]:
sql_statement= '''
SELECT
    C.CUST_LIFESTAGE,
    D.STORE_REGION,
    E.BASKET_DOMINANT_MISSION,
    SUM(A.SPEND) AS TOTAL_SPEND
FROM
    (
        SELECT
            CUST_CODE,
            STORE_CODE,
            BASKET_ID,
            SUM(SPEND) AS SPEND
        FROM
            transaction_details
        GROUP BY
            CUST_CODE,
            STORE_CODE,
            BASKET_ID
    ) AS A
LEFT JOIN Customer_details AS C ON A.CUST_CODE = C.CUST_CODE
LEFT JOIN Store_details AS D ON A.STORE_CODE = D.STORE_CODE
LEFT JOIN basket_details AS E ON A.BASKET_ID = E.BASKET_ID
GROUP BY
    C.CUST_LIFESTAGE,
    D.STORE_REGION,
    E.BASKET_DOMINANT_MISSION;
'''


df = pd.read_sql_query(f"EXPLAIN {sql_statement}", conn_normalized)

# Calculate the cost for each row
df["cost"] = df["opcode"].apply(lambda opcode: cost_estimates.get(opcode, 0))

# Calculate the overall cost
total_cost = df["cost"].sum()

# Print the overall cost
print(f"\033[1mTotal cost:\033[0m {total_cost}")


print("\033[1mExecution Plan\033[0m")
# Connect to the SQLite database
conn = sqlite3.connect('normalized.db')
cursor = conn.cursor()

# Execute EXPLAIN QUERY PLAN
explain_query = f"EXPLAIN QUERY PLAN {sql_statement}"
cursor.execute(explain_query)

# Fetch and print the execution plan
execution_plan = cursor.fetchall()
for plan in execution_plan:
    print(plan)

# Close the cursor and connection
cursor.close()
conn.close()

print("\033[1mExecution Time\033[0m")

# Connect to the SQLite database
conn = sqlite3.connect('normalized.db')
cursor = conn.cursor()

# Start timing
start_time = time.time()

cursor.execute(sql_statement)

# Get all results
results = cursor.fetchall()

# Stop timing
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time

# Close the cursor and connection
cursor.close()
conn.close()

# Print the execution time
print("Execution time:", execution_time, "seconds")


Total cost: 74
Execution Plan
(2, 0, 0, 'CO-ROUTINE A')
(8, 2, 0, 'SCAN transaction_details')
(10, 2, 0, 'USE TEMP B-TREE FOR GROUP BY')
(62, 0, 0, 'SCAN A')
(65, 0, 0, 'SEARCH C USING INDEX sqlite_autoindex_Customer_details_1 (CUST_CODE=?) LEFT-JOIN')
(73, 0, 0, 'SEARCH D USING INDEX sqlite_autoindex_Store_details_1 (STORE_CODE=?) LEFT-JOIN')
(81, 0, 0, 'SEARCH E USING INDEX sqlite_autoindex_basket_details_1 (BASKET_ID=?) LEFT-JOIN')
(89, 0, 0, 'USE TEMP B-TREE FOR GROUP BY')
Execution Time
Execution time: 1.0957579612731934 seconds


1. `(2, 0, 0, 'CO-ROUTINE A')`: This step indicates the beginning of a coroutine named A.

2. `(8, 2, 0, 'SCAN transaction_details')`: This step involves scanning the `transaction_details` table.

3. `(10, 2, 0, 'USE TEMP B-TREE FOR GROUP BY')`: Temporary B-tree usage for the GROUP BY operation. This indicates that a temporary data structure is used to facilitate the grouping operation on the `transaction_details` table.

4. `(62, 0, 0, 'SCAN A')`: This step involves scanning the result of the coroutine A.

5. `(65, 0, 0, 'SEARCH C USING INDEX sqlite_autoindex_Customer_details_1 (CUST_CODE=?) LEFT-JOIN')`: This involves searching in table C (Customer_details) using an index (sqlite_autoindex_Customer_details_1) on the column `CUST_CODE`. It appears to be part of a LEFT JOIN operation.

6. `(73, 0, 0, 'SEARCH D USING INDEX sqlite_autoindex_Store_details_1 (STORE_CODE=?) LEFT-JOIN')`: Similar to the previous step, this involves searching in table D (Store_details) using an index (sqlite_autoindex_Store_details_1) on the column `STORE_CODE`.

7. `(81, 0, 0, 'SEARCH E USING INDEX sqlite_autoindex_basket_details_1 (BASKET_ID=?) LEFT-JOIN')`: Similar to the previous steps, this involves searching in table E (basket_details) using an index (sqlite_autoindex_basket_details_1) on the column `BASKET_ID`.

8. `(89, 0, 0, 'USE TEMP B-TREE FOR GROUP BY')`: Temporary B-tree usage for the GROUP BY operation. This indicates that a temporary data structure is used to facilitate the grouping operation on the result of the coroutine A.


### - We can observe that the optimized code had reduced the code's exectuion time to 28.53% 
### - Eventhough the cost has increased there is significant reduction in execution time 
### - Optimization always is trade off between cost and time (resource utilization and query performance)

# Query 3

### Unoptimized query

Existing problem:
   - In this unoptimized query, multiple joins, subqueries, and conditions are involved, potentially leading to high computational cost

In [311]:
sql_statement ='''

SELECT 
*
FROM 
(
SELECT
    A.CUST_CODE,
    B.CUST_LIFESTAGE,
    C.STORE_REGION,
    D.BASKET_DOMINANT_MISSION,
    E.PROD_CODE_40,
    F.BASKET_SIZE,
    G.SUM_QUANTITY,
    SUBSTR(SHOP_DATE, 1, 4) || '-' || SUBSTR(SHOP_DATE, 5, 2) || '-' || SUBSTR(SHOP_DATE, 7, 2) AS SHOP_DATE
FROM
    transaction_details AS A
LEFT JOIN Customer_details AS B ON A.CUST_CODE = B.CUST_CODE
LEFT JOIN Store_details AS C ON A.STORE_CODE = C.STORE_CODE
LEFT JOIN basket_details AS D ON A.BASKET_ID = D.BASKET_ID
LEFT JOIN product_40_code AS E ON A.PROD_CODE = E.PROD_CODE_40
LEFT JOIN basket_details AS F ON A.BASKET_ID = F.BASKET_ID
LEFT JOIN (
    SELECT BASKET_ID, SUM(QUANTITY) AS SUM_QUANTITY
    FROM transaction_details
    GROUP BY BASKET_ID
) AS G ON A.BASKET_ID = G.BASKET_ID
)
WHERE 
DATE(SHOP_DATE) >= '2007-02-01'
AND 
DATE(SHOP_DATE) <= '2007-12-31'

ORDER BY
    SUM_QUANTITY DESC;
'''

df = pd.read_sql_query(f"EXPLAIN {sql_statement}", conn_normalized)
df



# Calculate the cost for each row
df["cost"] = df["opcode"].apply(lambda opcode: cost_estimates.get(opcode, 0))

# Calculate the overall cost
total_cost = df["cost"].sum()

# Print the overall cost
print(f"\033[1mTotal cost:\033[0m {total_cost}")


print("\033[1mExecution Plan\033[0m")
# Connect to the SQLite database
conn = sqlite3.connect('normalized.db')
cursor = conn.cursor()

# Execute EXPLAIN QUERY PLAN
explain_query = f"EXPLAIN QUERY PLAN {sql_statement}"
cursor.execute(explain_query)

# Fetch and print the execution plan
execution_plan = cursor.fetchall()
for plan in execution_plan:
    print(plan)

# Close the cursor and connection
cursor.close()
conn.close()

print("\033[1mExecution Time\033[0m")

# Connect to the SQLite database
conn = sqlite3.connect('normalized.db')
cursor = conn.cursor()

# Start timing
start_time = time.time()

cursor.execute(sql_statement)

# Get all results
results = cursor.fetchall()

# Stop timing
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time

# Close the cursor and connection
cursor.close()
conn.close()

# Print the execution time
print("Execution time:", execution_time, "seconds")



Total cost: 122
Execution Plan
(3, 0, 0, 'MATERIALIZE G')
(10, 3, 0, 'SCAN transaction_details')
(12, 3, 0, 'USE TEMP B-TREE FOR GROUP BY')
(62, 0, 0, 'SCAN A')
(88, 0, 0, 'SEARCH B USING INDEX sqlite_autoindex_Customer_details_1 (CUST_CODE=?) LEFT-JOIN')
(95, 0, 0, 'SEARCH C USING INDEX sqlite_autoindex_Store_details_1 (STORE_CODE=?) LEFT-JOIN')
(102, 0, 0, 'SEARCH D USING INDEX sqlite_autoindex_basket_details_1 (BASKET_ID=?) LEFT-JOIN')
(119, 0, 0, 'SEARCH E USING AUTOMATIC COVERING INDEX (PROD_CODE_40=?) LEFT-JOIN')
(126, 0, 0, 'SEARCH F USING INDEX sqlite_autoindex_basket_details_1 (BASKET_ID=?) LEFT-JOIN')
(146, 0, 0, 'SEARCH G USING AUTOMATIC COVERING INDEX (BASKET_ID=?) LEFT-JOIN')
(197, 0, 0, 'USE TEMP B-TREE FOR ORDER BY')
Execution Time
Execution time: 0.5786621570587158 seconds


1. `(3, 0, 0, 'MATERIALIZE G')`: This step involves materializing the result of the subquery G. Materialization means creating a temporary table to store the results of the subquery.

2. `(10, 3, 0, 'SCAN transaction_details')`: This step involves scanning the `transaction_details` table.

3. `(12, 3, 0, 'USE TEMP B-TREE FOR GROUP BY')`: Temporary B-tree usage for the GROUP BY operation on the `transaction_details` table.

4. `(62, 0, 0, 'SCAN A')`: This step involves scanning the result of the subquery A.

5. `(88, 0, 0, 'SEARCH B USING INDEX sqlite_autoindex_Customer_details_1 (CUST_CODE=?) LEFT-JOIN')`: Searching in the `Customer_details` table (B) using an index (sqlite_autoindex_Customer_details_1) on the column `CUST_CODE` for LEFT JOIN.

6. `(95, 0, 0, 'SEARCH C USING INDEX sqlite_autoindex_Store_details_1 (STORE_CODE=?) LEFT-JOIN')`: Searching in the `Store_details` table (C) using an index (sqlite_autoindex_Store_details_1) on the column `STORE_CODE` for LEFT JOIN.

7. `(102, 0, 0, 'SEARCH D USING INDEX sqlite_autoindex_basket_details_1 (BASKET_ID=?) LEFT-JOIN')`: Searching in the `basket_details` table (D) using an index (sqlite_autoindex_basket_details_1) on the column `BASKET_ID` for LEFT JOIN.

8. `(119, 0, 0, 'SEARCH E USING AUTOMATIC COVERING INDEX (PROD_CODE_40=?) LEFT-JOIN')`: Searching in the `product_40_code` table (E) using an automatic covering index on the column `PROD_CODE_40` for LEFT JOIN.

9. `(126, 0, 0, 'SEARCH F USING INDEX sqlite_autoindex_basket_details_1 (BASKET_ID=?) LEFT-JOIN')`: Searching in the `basket_details` table (F) using an index (sqlite_autoindex_basket_details_1) on the column `BASKET_ID` for LEFT JOIN.

10. `(146, 0, 0, 'SEARCH G USING AUTOMATIC COVERING INDEX (BASKET_ID=?) LEFT-JOIN')`: Searching in the materialized result of subquery G using an automatic covering index on the column `BASKET_ID` for LEFT JOIN.

11. `(197, 0, 0, 'USE TEMP B-TREE FOR ORDER BY')`: Temporary B-tree usage for the ORDER BY operation.



### Optmized Query

Solution:
- Optimizations include simplifying the WHERE condition, aggregating data at an early stage, and avoiding unnecessary columns in the SELECT clause
- Ensure that relevant indexes are in place, particularly on columns used in JOIN and WHERE conditions, to improve performance

In [312]:
sql_statement ='''

SELECT
    B.CUST_LIFESTAGE,
    C.STORE_REGION,
    D.BASKET_DOMINANT_MISSION,
    E.PROD_CODE_40,
    F.BASKET_SIZE,
    G.SUM_QUANTITY
FROM
    transaction_details AS A
LEFT JOIN Customer_details AS B ON A.CUST_CODE = B.CUST_CODE
LEFT JOIN Store_details AS C ON A.STORE_CODE = C.STORE_CODE
LEFT JOIN basket_details AS D ON A.BASKET_ID = D.BASKET_ID
LEFT JOIN product_40_code AS E ON A.PROD_CODE = E.PROD_CODE_40
LEFT JOIN basket_details AS F ON A.BASKET_ID = F.BASKET_ID
LEFT JOIN (
    SELECT BASKET_ID, SUM(QUANTITY) AS SUM_QUANTITY
    FROM transaction_details
    GROUP BY BASKET_ID
) AS G ON A.BASKET_ID = G.BASKET_ID
WHERE
    DATE(SHOP_DATE) >= '2007-02-01'
    AND 
    DATE(SHOP_DATE) <= '2007-12-31'
ORDER BY
    G.SUM_QUANTITY DESC;

'''

df = pd.read_sql_query(f"EXPLAIN {sql_statement}", conn_normalized)
df



# Calculate the cost for each row
df["cost"] = df["opcode"].apply(lambda opcode: cost_estimates.get(opcode, 0))

# Calculate the overall cost
total_cost = df["cost"].sum()

# Print the overall cost
print(f"\033[1mTotal cost:\033[0m {total_cost}")


print("\033[1mExecution Plan\033[0m")
# Connect to the SQLite database
conn = sqlite3.connect('normalized.db')
cursor = conn.cursor()

# Execute EXPLAIN QUERY PLAN
explain_query = f"EXPLAIN QUERY PLAN {sql_statement}"
cursor.execute(explain_query)

# Fetch and print the execution plan
execution_plan = cursor.fetchall()
for plan in execution_plan:
    print(plan)

# Close the cursor and connection
cursor.close()
conn.close()

print("\033[1mExecution Time\033[0m")

# Connect to the SQLite database
conn = sqlite3.connect('normalized.db')
cursor = conn.cursor()

# Start timing
start_time = time.time()

cursor.execute(sql_statement)

# Get all results
results = cursor.fetchall()

# Stop timing
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time

# Close the cursor and connection
cursor.close()
conn.close()

# Print the execution time
print("Execution time:", execution_time, "seconds")



Total cost: 122
Execution Plan
(3, 0, 0, 'MATERIALIZE G')
(10, 3, 0, 'SCAN transaction_details')
(12, 3, 0, 'USE TEMP B-TREE FOR GROUP BY')
(62, 0, 0, 'SCAN A')
(70, 0, 0, 'SEARCH B USING INDEX sqlite_autoindex_Customer_details_1 (CUST_CODE=?) LEFT-JOIN')
(77, 0, 0, 'SEARCH C USING INDEX sqlite_autoindex_Store_details_1 (STORE_CODE=?) LEFT-JOIN')
(84, 0, 0, 'SEARCH D USING INDEX sqlite_autoindex_basket_details_1 (BASKET_ID=?) LEFT-JOIN')
(101, 0, 0, 'SEARCH E USING AUTOMATIC COVERING INDEX (PROD_CODE_40=?) LEFT-JOIN')
(108, 0, 0, 'SEARCH F USING INDEX sqlite_autoindex_basket_details_1 (BASKET_ID=?) LEFT-JOIN')
(128, 0, 0, 'SEARCH G USING AUTOMATIC COVERING INDEX (BASKET_ID=?) LEFT-JOIN')
(168, 0, 0, 'USE TEMP B-TREE FOR ORDER BY')
Execution Time
Execution time: 0.03756904602050781 seconds


1. `(3, 0, 0, 'MATERIALIZE G')`: Materializing the result of the subquery G. Materialization involves creating a temporary table to store the results of the subquery.

2. `(10, 3, 0, 'SCAN transaction_details')`: Scanning the `transaction_details` table.

3. `(12, 3, 0, 'USE TEMP B-TREE FOR GROUP BY')`: Utilizing a temporary B-tree for the GROUP BY operation on the `transaction_details` table.

4. `(62, 0, 0, 'SCAN A')`: Scanning the result of subquery A.

5. `(70, 0, 0, 'SEARCH B USING INDEX sqlite_autoindex_Customer_details_1 (CUST_CODE=?) LEFT-JOIN')`: Searching in the `Customer_details` table (B) using an index (sqlite_autoindex_Customer_details_1) on the column `CUST_CODE` for LEFT JOIN.

6. `(77, 0, 0, 'SEARCH C USING INDEX sqlite_autoindex_Store_details_1 (STORE_CODE=?) LEFT-JOIN')`: Searching in the `Store_details` table (C) using an index (sqlite_autoindex_Store_details_1) on the column `STORE_CODE` for LEFT JOIN.

7. `(84, 0, 0, 'SEARCH D USING INDEX sqlite_autoindex_basket_details_1 (BASKET_ID=?) LEFT-JOIN')`: Searching in the `basket_details` table (D) using an index (sqlite_autoindex_basket_details_1) on the column `BASKET_ID` for LEFT JOIN.

8. `(101, 0, 0, 'SEARCH E USING AUTOMATIC COVERING INDEX (PROD_CODE_40=?) LEFT-JOIN')`: Searching in the `product_40_code` table (E) using an automatic covering index on the column `PROD_CODE_40` for LEFT JOIN.

9. `(108, 0, 0, 'SEARCH F USING INDEX sqlite_autoindex_basket_details_1 (BASKET_ID=?) LEFT-JOIN')`: Searching in the `basket_details` table (F) using an index (sqlite_autoindex_basket_details_1) on the column `BASKET_ID` for LEFT JOIN.

10. `(128, 0, 0, 'SEARCH G USING AUTOMATIC COVERING INDEX (BASKET_ID=?) LEFT-JOIN')`: Searching in the materialized result of subquery G using an automatic covering index on the column `BASKET_ID` for LEFT JOIN.

11. `(168, 0, 0, 'USE TEMP B-TREE FOR ORDER BY')`: Utilizing a temporary B-tree for the ORDER BY operation.


### We can observe that the optimized code had reduced the execution time to 83.53% 